In [ ]:
import os
import glob
import pandas as pd

# Caminho da pasta com os arquivos
caminho_pasta = r'C:\Users\zirn1\OneDrive\Desktop\Projetos\Analise_vinicultura\Dataset_INMET'

# Lista todos os arquivos .csv da pasta
arquivos_csv = glob.glob(os.path.join(caminho_pasta, '*.csv'))

# Lista para armazenar os DataFrames
dfs = []

# Loop pelos arquivos
for arquivo in arquivos_csv:
    try:
        with open(arquivo, 'r', encoding='utf-8') as f:
            linhas = f.readlines()

        # Extrair os metadados
        metadados = {}
        linha_header = None
        for i, linha in enumerate(linhas):
            if linha.strip() == "":
                continue
            if linha.startswith("Data Medicao"):
                linha_header = i
                break
            if ":" in linha:
                chave, valor = linha.strip().split(":", 1)
                metadados[chave.strip()] = valor.strip()

        # Verifica se encontrou o cabeçalho
        if linha_header is None:
            print(f"⚠️ Cabeçalho não encontrado em {arquivo}, pulando arquivo.")
            continue

        # Carregar a tabela de dados
        df = pd.read_csv(arquivo, sep=";", skiprows=linha_header, encoding='utf-8')

        # Adiciona os metadados
        for chave, valor in metadados.items():
            df[chave] = valor

        dfs.append(df)

    except Exception as e:
        print(f"❌ Erro ao processar {arquivo}: {e}")

# Concatenar os DataFrames
if dfs:
    df_geral = pd.concat(dfs, ignore_index=True)
    print("✅ Dados concatenados com sucesso!")
else:
    print("⚠️ Nenhum DataFrame carregado. Verifique os arquivos na pasta.")


In [ ]:
print(df_geral.columns)


Index(['Data Medicao', 'PRECIPITACAO TOTAL, MENSAL (AUT)(mm)',
       'PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)',
       'TEMPERATURA MEDIA, MENSAL (AUT)(°C)',
       'VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)', 'Unnamed: 5', 'Nome',
       'Codigo Estacao', 'Latitude', 'Longitude', 'Altitude', 'Situacao',
       'Data Inicial', 'Data Final', 'Periodicidade da Medicao'],
      dtype='object')


In [ ]:
df_geral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15647 entries, 0 to 15646
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Data Medicao                                 15647 non-null  object 
 1   PRECIPITACAO TOTAL, MENSAL (AUT)(mm)         12344 non-null  object 
 2   PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)  13516 non-null  object 
 3   TEMPERATURA MEDIA, MENSAL (AUT)(°C)          13806 non-null  object 
 4   VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)    12940 non-null  object 
 5   Unnamed: 5                                   0 non-null      float64
 6   Nome                                         15647 non-null  object 
 7   Codigo Estacao                               15647 non-null  object 
 8   Latitude                                     15647 non-null  object 
 9   Longitude                                    15647 non-null  object 
 10

In [ ]:
# Etapa 1: Renomear colunas
df_geral.rename(columns={
    'Data Medicao': 'Data',
    'PRECIPITACAO TOTAL, MENSAL (AUT)(mm)': 'Precipitacao_mm',
    'PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)': 'Pressao_mB',
    'TEMPERATURA MEDIA, MENSAL (AUT)(°C)': 'Temperatura_C',
    'VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)': 'Vento_m_s'
}, inplace=True)

# Etapa 2: Remover colunas desnecessárias
colunas_remover = [
    'Unnamed: 5', 'Codigo Estacao', 'Latitude', 'Longitude',
    'Altitude', 'Situacao', 'Data Inicial', 'Data Final',
    'Periodicidade da Medicao', 'Pressao_mB'
]
df_geral.drop(columns=[col for col in colunas_remover if col in df_geral.columns], inplace=True)

# Etapa 3: Converter colunas climáticas para float
df_geral['Precipitacao_mm'] = pd.to_numeric(df_geral['Precipitacao_mm'], errors='coerce')
df_geral['Temperatura_C'] = pd.to_numeric(df_geral['Temperatura_C'], errors='coerce')
df_geral['Vento_m_s'] = pd.to_numeric(df_geral['Vento_m_s'], errors='coerce')

# Etapa 4: Converter datas
df_geral['Data'] = pd.to_datetime(df_geral['Data'], errors='coerce')
df_geral.dropna(subset=['Data'], inplace=True)

# Etapa 5: Criar colunas de Ano e Mês
df_geral['Ano'] = df_geral['Data'].dt.year
df_geral['Mes'] = df_geral['Data'].dt.month

# Etapa 6: Agrupar por Ano e Mês e gerar estatísticas
df_resumo = df_geral.groupby(['Ano', 'Mes']).agg(
    Precipitacao_min=('Precipitacao_mm', 'min'),
    Precipitacao_media=('Precipitacao_mm', 'mean'),
    Precipitacao_max=('Precipitacao_mm', 'max'),
    
    Temperatura_min=('Temperatura_C', 'min'),
    Temperatura_media=('Temperatura_C', 'mean'),
    Temperatura_max=('Temperatura_C', 'max'),
    
    Vento_min=('Vento_m_s', 'min'),
    Vento_media=('Vento_m_s', 'mean'),
    Vento_max=('Vento_m_s', 'max')
).reset_index()

# Etapa 7: Salvar o CSV final
caminho_saida = r'C:\Users\zirn1\OneDrive\Desktop\Projetos\Analise_vinicultura\resumo_climatico_mensal.csv'
df_resumo.to_csv(caminho_saida, index=False)
print(f"✅ Resumo climático mensal salvo com sucesso em:\n{caminho_saida}")


✅ Resumo climático mensal salvo com sucesso em:
C:\Users\zirn1\OneDrive\Desktop\Projetos\Analise_vinicultura\resumo_climatico_mensal.csv


In [ ]:
# 1. Remover linhas com valores NaN
df_resumo = df_resumo.dropna()

# 2. Arredondar e formatar os números com 2 casas decimais
colunas_formatar = [
    'Precipitacao_min', 'Precipitacao_media', 'Precipitacao_max',
    'Temperatura_min', 'Temperatura_media', 'Temperatura_max',
    'Vento_min', 'Vento_media', 'Vento_max'
]

# Arredondar os dados para 2 casas decimais
df_resumo[colunas_formatar] = df_resumo[colunas_formatar].round(2)

# 3. Salvar o DataFrame limpo e formatado
caminho_saida_formatado = r'C:\Users\zirn1\OneDrive\Desktop\Projetos\Analise_vinicultura\resumo_climatico_mensal_formatado.csv'
df_resumo.to_csv(caminho_saida_formatado, index=False)

print("✅ Resumo climático limpo e formatado salvo com sucesso em:")
print(caminho_saida_formatado)

# 4. Exibir amostra formatada
print("\n📊 Exemplo de dados formatados:")
print(df_resumo.head(10).to_string(index=False))


✅ Resumo climático limpo e formatado salvo com sucesso em:
C:\Users\zirn1\OneDrive\Desktop\Projetos\Analise_vinicultura\resumo_climatico_mensal_formatado.csv

📊 Exemplo de dados formatados:
 Ano  Mes  Precipitacao_min  Precipitacao_media  Precipitacao_max  Temperatura_min  Temperatura_media  Temperatura_max  Vento_min  Vento_media  Vento_max
2010    1               0.0              193.20             357.0             22.0              23.83             26.0        2.0         2.43        3.0
2010    2               0.0              142.00             303.0             21.0              23.86             26.0        0.0         2.25        5.0
2010    3               0.0              137.07             374.0             19.0              21.20             25.0        0.0         2.44        6.0
2010    4               0.0              156.27             339.0             19.0              19.60             21.0        0.0         2.00        3.0
2010    5               0.0             

C:\Users\zirn1\AppData\Local\Temp\ipykernel_9400\2234350399.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resumo[colunas_formatar] = df_resumo[colunas_formatar].round(2)


Data               0
Precipitacao_mm    0
Pressao_mB         0
Temperatura_C      0
Vento_m_s          0
Nome               0
Codigo Estacao     0
Latitude           0
Longitude          0
Altitude           0
dtype: int64
(11763, 10)


Data               0
Precipitacao_mm    0
Pressao_mB         0
Temperatura_C      0
Vento_m_s          0
Nome               0
Codigo Estacao     0
Latitude           0
Longitude          0
Altitude           0
Ano                0
Mes                0
dtype: int64


,Data,Precipitacao_mm,Pressao_mB,Temperatura_C,Vento_m_s,Latitude,Longitude,Altitude,Ano,Mes
count,11763,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000
mean,2017-08-29 23:54:36.817138688,141.049987,960.141928,19.148338,2.396566,-27.759131,-52.021000,484.877159,2017.128454,6.444189
min,2010-01-31 00:00:00,0.000000,818.100000,6.000000,0.000000,-33.742222,-57.081944,2.000000,2010.000000,1.000000
25%,2013-09-30 00:00:00,72.200000,925.900000,16.200000,1.600000,-29.350278,-53.514167,106.990000,2013.000000,3.000000
50%,2017-09-30 00:00:00,123.000000,962.300000,19.400000,2.400000,-27.890556,-52.167778,455.270000,2017.000000,6.000000
75%,2021-04-30 00:00:00,190.600000,1002.200000,22.300000,3.100000,-26.248611,-50.180556,799.580000,2021.000000,10.000000
max,2025-05-31 00:00:00,773.800000,1024.800000,29.500000,10.100000,-22.639444,-48.326111,1790.380000,2025.000000,12.000000
std,NaN,94.896545,42.481842,3.993115,1.265062,2.367624,2.036295,384.188260,4.438349,3.528109


,Ano,Mes,Precipitacao_mm,Temperatura_C,Vento_m_s
0,2010,1,218.093548,23.116129,2.451613
1,2010,2,160.442623,23.965574,2.485246
2,2010,3,147.028571,21.912698,2.276190
3,2010,4,161.863492,18.838095,2.519048
4,2010,5,158.652459,15.316393,2.526230
...,...,...,...,...,...
180,2025,1,105.820339,23.128814,2.223729
181,2025,2,124.992453,24.492453,1.973585
182,2025,3,70.189474,22.921053,2.014035
183,2025,4,111.327273,18.352727,1.876364


    Ano  Temperatura_C_mean  Temperatura_C_max  Temperatura_C_min  \
0  2010           18.501783          23.965574          14.024194   
1  2011           18.384845          23.600000          13.126471   
2  2012           19.411863          23.961404          13.170833   
3  2013           18.410761          23.233824          13.533846   
4  2014           19.606506          24.127692          14.986792   

   Precipitacao_mm_mean  Precipitacao_mm_max  Precipitacao_mm_min  \
0            138.600572           218.093548            48.495082   
1            143.197044           223.630508            63.597059   
2            119.029414           207.097222            43.679452   
3            141.005341           188.575000            84.083077   
4            157.748645           233.142857            72.447458   

   Vento_m_s_mean  Vento_m_s_max  Vento_m_s_min  
0        2.657370       3.006557       2.276190  
1        2.711281       3.330508       2.246032  
2        2.573658   